<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

#  **<font color="#76b900">노트북 6: 디코더 전용 모델 기반 텍스트 생성</font>**

텍스트를 순차적으로 생성하는 작업(예: 스토리 생성, 자유 응답, 코드 생성 등)에는 **디코더 전용 모델**이 적합합니다.  
대표적인 예는 **GPT-2**이며, 이는 인코더 없이 디코더만으로 구성되어 있습니다.

이 노트북에서는 다음을 다룹니다:


###  학습 목표
- 디코더 전용 모델의 구조 이해  
- 자가회귀 방식의 텍스트 생성 원리 학습  
- GPT-2 모델을 이용한 자유 텍스트 생성 실습  
- Zero-shot/Prompt 기반 추론의 개념 도입  
- 생성 품질을 제어하는 매개변수의 역할 이해


### 주요 학습 내용
- Auto-Regressive(자가회귀) 생성 구조 설명  
- GPT 시리즈의 디코더 전용 아키텍처  
- 입력 시퀀스를 기반으로 한 토큰 순차 예측 과정  
- 다양한 프롬프트 및 설정에 따른 결과 비교 실습  
- 생성 품질 조절 매개변수: `temperature`, `top_k`, `top_p`

## **Part 1: GPT 스타일 모델의 개념 정리**

**BERT**와 같은 인코더 전용 모델은 “이해”는 잘하지만 “생성” 능력은 부족합니다.

**GPT-2, GPT-3, GPT-4** 등은 Transformer의 **디코더**만으로 구성된 **디코더 전용 모델**입니다.

- **자기회귀(Autoregressive)** 방식:  
- 입력: 이전까지의 모든 토큰  
- 출력: 다음 토큰 하나 예측  
- 반복하며 문장을 완성


### **자기회귀 생성 예시**

입력 프롬프트:
```
Once upon a time
```

GPT가 생성한 출력 예시:
```
Once upon a time, there was a young princess who dreamed of exploring the world beyond her castle walls.
```


#### **디코더 구조의 장점 요약**

| 특성             | 설명 |
|------------------|------|
| 순차적 생성      | 이전 단어 기반 다음 단어 생성 |
| 유연한 생성      | 프롬프트로 다양한 응답 가능 |
| 고정된 출력 없음 | 소설, 대화, 코드 등 모두 생성 |
| 응용 분야        | 챗봇, 요약, 글쓰기 보조 등   |

아래는 실제 GPT-2 디코더 모델로 텍스트를 여러 개 생성하는 코드입니다.

In [ ]:
from transformers import pipeline
import warnings
warnings.filterwarnings("ignore")

# GPT-2 모델을 사용하여 텍스트 생성 파이프라인을 생성합니다
# pipeline()은 transformers 라이브러리에서 제공하는 편리한 추론 도구입니다
# 'text-generation' 태스크와 'gpt2' 모델을 지정하여 텍스트 생성기를 초기화합니다
generator = pipeline('text-generation', model='gpt2', pad_token_id=50256)
generator.tokenizer.pad_token_id = 50256

# "Hello world"로 시작하는 텍스트를 5개 생성합니다
# max_length=20: 생성될 각 텍스트의 최대 길이를 20 토큰으로 제한합니다
# num_return_sequences=5: 서로 다른 텍스트 시퀀스를 5개 생성합니다
# 이는 다양한 텍스트 생성 결과를 얻기 위한 것입니다
generator("Hello world,", max_length=20, num_return_sequences=5)

###  **2-1. 생성 품질 조절 실험**

| 파라미터      | 역할 |
|---------------|------|
| temperature   | 샘플링 분산 조절 (값이 높을수록 더 다양한 결과 생성) |
| top_k         | 확률 상위 K개의 단어만 후보로 고려 |
| top_p         | 누적 확률 상위 P%까지 후보로 고려 (nucleus sampling) |
| max_length    | 생성될 전체 텍스트의 최대 토큰 수 (입력 + 출력 포함) |

In [ ]:
generator("AI will change the world", 
          max_length=30, 
          temperature=1.0, 
          top_k=50, 
          top_p=0.95, 
          num_return_sequences=3)

#### Part 4: 다양한 프롬프트 실험
- 다양한 작업에 하나의 모델로 접근 가능 (멀티태스크 특성)

In [ ]:
prompts = [
    "Translate English to French: I am happy.", # 번역 (Translation) 
    "Write a poem about AI and nature.", # 시 생성 (Poetry Generation)
    "Generate a Python function that calculates Fibonacci numbers.", # 코드 생성 (Code Generation)
    "Explain why the sky is blue in simple terms." # 설명 생성 (Explanation Generation)
]

for p in prompts:
    print(f"\nPrompt: {p}")
    print(generator(p, max_length=50)[0]["generated_text"])

### 디코더 전용 모델의 장점과 한계 비교

| 항목 | 장점 | 한계 |
|------|------|------|
| 생성 방식 | 이전 토큰 기반 자연스러운 순차 생성 | 입력 전체를 요약하거나 이해하는 데는 부적합 |
| 응용 범위 | 소설, 코드, 챗봇 등 자유 생성 | QA, 번역처럼 복잡한 문맥은 처리 어려움 |
| 유연성 | 다양한 프롬프트에 반응 가능 | 항상 왼쪽 → 오른쪽 방향만 고려 |
| 계산 효율성 | 디코더만 사용하므로 상대적으로 구조 단순 | 긴 시퀀스 생성 시 연산량 증가, 느려짐 |



### 디코더 모델이 적절한 사용 사례와 부적절한 사례

| 작업 유형 | 디코더 전용 적합? | 이유 |
|-----------|------------------|------|
| 소설 생성 | ✅ 적합 | 프롬프트 기반 시퀀스 확장 |
| 코드 생성 | ✅ 적합 | 정형 패턴 기반 순차 생성 |
| 챗봇 응답 | ✅ 적합 | 전 대화문맥 기반 생성 |
| 기계 번역 | ❌ 부적합 | 입력 전체 요약 필요 → 인코더 필요 |
| 문장 완성 (중간 단어 예측) | ❌ 부적합 | 양방향 문맥 활용 필요 → BERT 구조 적합 |



### GPT 디코더 블록 내부 구조 요약

디코더는 N개의 동일한 블록이 쌓여 있는 구조로, 각 블록은 다음과 같은 단계를 포함합니다:

```text
[입력 토큰]
→ [토큰 임베딩 + 포지셔널 인코딩]
→ 반복 N회:
   - Masked Self-Attention
   - Add & LayerNorm
   - Feed-Forward Network (FFN)
   - Add & LayerNorm
→ [출력 로짓 → Softmax → 다음 토큰 예측]
```



### 대규모 생성 모델 분석 및 배포 시 고려 사항

| 모델 | 파라미터 수 | GPU 메모리 | 주요 특성 |
|------|--------------|-------------|------------|
| GPT-2 | ~1.5B | 4GB | 로컬에서도 실행 가능 |
| GPT-3 | 175B | API Only | 공개 모델 X, OpenAI API 필요 |
| GPT-4 | 비공개 | API Only | 멀티모달 확장됨 |

- **배포 기술 예시**:
  - LoRA: 파라미터 일부만 미세조정
  - GPTQ: 4bit 양자화로 VRAM 절약
  - 모델 병렬화: 여러 GPU에 나눠서 로딩

### 연습문제: 생성 품질과 모델 한계 실험

아래 조건을 만족하는 코드를 작성하고, 출력 결과를 통해 디코더 전용 모델의 특성과 한계를 직접 체감해보세요.

**[문제]**

1. `"The impact of artificial intelligence on society"`라는 프롬프트를 사용하여,
2. `temperature=0.7`, `top_k=40`, `top_p=0.95`, `max_length=50`으로 설정해 텍스트를 2개 생성합니다.
3. 그다음, `temperature=1.5`, `top_k=0`, `top_p=1.0`으로 같은 프롬프트로 텍스트 2개를 다시 생성합니다.
4. 결과를 비교하면서 다음 두 질문에 대한 관찰 결과를 출력하세요:
   - (1) 생성된 문장이 일관적이고 논리적인가?
   - (2) 주제에서 벗어나는 정도는 어떤가?

<details>
<summary><b>✅ 정답 보기 (예시)</b></summary>

```python
# 여기에 generator 설정 코드가 있다고 가정

prompt = "The impact of artificial intelligence on society"

# 설정 1: 안정적 출력
output_1 = generator(prompt, temperature=0.7, top_k=40, top_p=0.95, max_length=50, num_return_sequences=2)

# 설정 2: 매우 다양한 출력
output_2 = generator(prompt, temperature=1.5, top_k=0, top_p=1.0, max_length=50, num_return_sequences=2)

print("=== 안정적 출력 ===")
print(output_1)

print("\n=== 다양성 중심 출력 ===")
print(output_2)
```
- **(1)** 낮은 temperature 설정에서는 비교적 일관된 문장과 주제 유지가 확인됨.  
- **(2)** 높은 temperature 설정에서는 더 창의적이고 다양한 문장이 생성되나, 종종 주제와 무관한 내용이 포함됨.  
- 이 실험은 하이퍼파라미터 조절이 얼마나 생성 품질에 큰 영향을 주는지를 체험할 수 있음.

</details>


# <font color="#76b900">**마무리**</font>

지금까지 우리는 디코더 전용 언어 모델(GPT 시리즈)이 이전 문맥을 기반으로  
순차적으로 텍스트를 생성하는 방식과, 다양한 프롬프트를 통해  
텍스트, 코드, 대화 등을 생성하는 능력을 살펴보았습니다.

이러한 모델들은 생성 중심 작업에 매우 유용하지만,  
복잡한 입력 구조나 문맥 전체의 압축이 필요한 태스크에는 적절하지 않을 수 있습니다.

또한, GPT-2 같은 비교적 작은 모델은 로컬에서도 실습 가능하지만,  
GPT-3/4와 같은 대규모 모델은 API 기반 접근만 가능하며,  
모델 크기 및 배포 환경에 따라 다양한 기술적 고려사항이 필요합니다.

다음 노트북에서는, 문맥 주입(Context Injection)을 활용하여 언어 모델을 확률적 앵무새(Stochastic Parrots)에서 한 단계 발전시키는 방법에 대해서 알아보겠습니다.

In [ ]:
## 작업이 끝났을 때 실행해주세요!

import IPython
app = IPython.Application.instance() # 현재 실행 중인 IPython 인스턴스(커널)를 가져옵니다.
# 커널을 안전하게 종료합니다.
app.kernel.do_shutdown(True)

<center><a href="https://www.nvidia.com/ko-kr/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>